<a href="https://colab.research.google.com/github/GAMZAMANDU/gukjeung24/blob/main/%E1%84%80%E1%85%AE%E1%86%A8%E1%84%92%E1%85%AC%E1%84%8B%E1%85%B4%E1%84%8B%E1%85%A1%E1%86%AB%E1%84%87%E1%85%AE%E1%86%AB%E1%84%89%E1%85%A5%E1%86%A8%ED%95%98%EB%8B%A4_%EA%B5%AD%EC%A0%9524.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 국회의안 데이터 분석 프로젝트 - 국정24

## 프로젝트 개요
본 프로젝트는 국회 Open API를 활용하여 22대 국회 의원들의 정책 분야별 활동을 분석하는 프로젝트입니다.

### 분석 목표
- 국회 의원별 정책 분야 집중도 분석한다
- 정책 분야별 활동 현황 파악한다
- 의원별 발의 의안 특성 분석한다

### 분석 과정
1. **데이터 전처리**: Open API 데이터 수집 및 정제
2. **데이터 분석**: 의원별 정책 분야 점수 계산
3. **데이터 시각화**: 히트맵을 통한 시각적 분석


# 1. 데이터 전처리 (Data Preprocessing)

## 1.1 필요한 라이브러리 임포트

In [2]:
import requests
import pandas as pd
import json
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import platform


plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False # 마이너스 폰트 깨짐 방지

import warnings
warnings.filterwarnings('ignore')

print("라이브러리 임포트 및 한글 폰트 설정 완료")

✅ 라이브러리 임포트 및 한글 폰트 설정 완료


## 1.2 API 설정 및 연결 테스트

In [ ]:
API_KEY = ""
BILL_URL = "https://open.assembly.go.kr/portal/openapi/nwbpacrgavhjryiph"
VOTE_URL = "https://open.assembly.go.kr/portal/openapi/ncocpgfiaoituanbr"
MEMBER_URL = "https://open.assembly.go.kr/portal/openapi/nwbpacrgavhjryiph"

def test_api():
    print("=== 국회 Open API 연결 테스트 ===")

    params = {
        'Key': API_KEY,
        'Type': 'json',
        'pIndex': 1,
        'pSize': 5,
        'AGE': '22'
    }

    try:
        response = requests.get(BILL_URL, params=params)
        print(f"API 응답 상태: {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            print("✅ API 연결 성공!")
            print(f"응답 데이터 구조 키들: {list(data.keys())}")
            return True, data
        else:
            print(f"❌ API 연결 실패: {response.text}")
            return False, None

    except Exception as e:
        print(f"❌ API 연결 중 오류 발생: {e}")
        return False, None

success, sample_data = test_api()

=== 국회 Open API 연결 테스트 ===
API 응답 상태: 200
✅ API 연결 성공!
응답 데이터 구조 키들: ['nwbpacrgavhjryiph']


## 1.3 데이터 수집 및 정제

In [4]:
def collect_bills(max_pages=3):
    print("=== 국회 의안 데이터 수집 시작 ===")
    all_bills = []

    for page in range(1, max_pages + 1):
        params = {
            'Key': API_KEY,
            'Type': 'json',
            'pIndex': page,
            'pSize': 100,
            'AGE': '22'
        }

        try:
            response = requests.get(BILL_URL, params=params)

            if response.status_code == 200:
                data = response.json()
                bills = data['nwbpacrgavhjryiph'][1]['row']
                all_bills.extend(bills)
                print(f"📄 페이지 {page}: {len(bills)}건 수집")
            else:
                print(f"❌ 페이지 {page} 수집 실패")
                break

        except Exception as e:
            print(f"❌ 페이지 {page} 수집 중 오류: {e}")
            break

    print(f"✅ 총 수집된 의안 데이터: {len(all_bills)}건")
    return all_bills

bills_data = collect_bills(max_pages=3)

=== 국회 의안 데이터 수집 시작 ===
📄 페이지 1: 100건 수집
📄 페이지 2: 100건 수집
📄 페이지 3: 100건 수집
✅ 총 수집된 의안 데이터: 300건


In [5]:
def extract_proposer(text):
    if pd.isna(text) or '의원' not in text:
        return None

    if '등' in text:
        main = text.split('등')[0].strip()
        if '의원' in main:
            return main
    else:
        if text.endswith('의원'):
            return text

    return None

def organize_bills(bills_data):
    df = pd.DataFrame(bills_data)
    print(f"📊 전체 의안 데이터: {len(df)}건")

    df['proposer'] = df['PROPOSER'].apply(extract_proposer)

    df_valid = df.dropna(subset=['proposer']).copy()
    print(f"📊 주발의자가 명확한 의안: {len(df_valid)}건")

    bill_counts = df_valid.groupby('proposer').size().to_dict()
    print(f"📊 개별 의원 수: {len(bill_counts)}명")

    active_names = df_valid.groupby('proposer').size()
    active_names = active_names[active_names >= 2].index.tolist()

    active_df = df_valid[df_valid['proposer'].isin(active_names)]
    print(f"📈 2건 이상 발의한 활발한 의원: {len(active_names)}명")

    active_members = {}
    for member in active_names:
        member_bills = active_df[active_df['proposer'] == member]
        active_members[member] = [
            {
                'bill_no': row['BILL_NO'],
                'bill_name': row['BILL_NM'],
                'propose_date': row['PROPOSE_DT'],
                'proc_result': row['PROC_RESULT_CD']
            }
            for _, row in member_bills.iterrows()
        ]

    return df, active_members, bill_counts, active_df

bills_df, active_members, bill_counts, active_df = organize_bills(bills_data)

=== pandas를 활용한 의원별 데이터 정리 ===
📊 전체 의안 데이터: 300건
📊 주발의자가 명확한 의안: 91건
📊 개별 의원 수: 73명
📈 2건 이상 발의한 활발한 의원: 15명


# 2. 데이터 분석 (Data Analysis)

## 2.1 기초 데이터 탐색

In [6]:
print("=== 데이터 개요 ===")
print(f"📊 총 수집된 의안: {len(bills_df)}건")
print(f"👥 전체 의원 수: {len(bill_counts)}명")
print(f"🎯 분석 대상 의원: {len(active_members)}명 (2건 이상 발의)")

print("\n=== 의원별 발의 현황 (상위 10명) ===")
top_proposers = active_df['proposer'].value_counts().head(10)
for i, (member, count) in enumerate(top_proposers.items(), 1):
    print(f"{i:2d}. {member}: {count}건")

print("\n=== 실제 의안 예시 ===")
sample_bills = bills_df[['BILL_NM', 'PROPOSER', 'PROPOSE_DT']].head(5)
for i, (_, row) in enumerate(sample_bills.iterrows(), 1):
    print(f"{i}. {row['BILL_NM']}")
    print(f"   제안자: {row['PROPOSER']}")
    print(f"   제안일: {row['PROPOSE_DT']}")
    print()

print("=== 추가 통계 정보 ===")
print(f"📈 의안 처리 현황:")
proc_counts = bills_df['PROC_RESULT_CD'].value_counts()
for result, count in proc_counts.head(5).items():
    print(f"- {result}: {count}건")

print(f"\n📅 연도별 의안 발의 현황:")
bills_df['year'] = pd.to_datetime(bills_df['PROPOSE_DT'], errors='coerce').dt.year
yearly_counts = bills_df['year'].value_counts().sort_index()
for year, count in yearly_counts.items():
    if pd.notna(year):
        print(f"- {int(year)}년: {count}건")

=== 데이터 개요 ===
📊 총 수집된 의안: 300건
👥 전체 의원 수: 73명
🎯 분석 대상 의원: 15명 (2건 이상 발의)

=== 의원별 발의 현황 (상위 10명) ===
 1. 김예지의원: 3건
 2. 정을호의원: 3건
 3. 백승아의원: 3건
 4. 민형배의원: 2건
 5. 박용갑의원: 2건
 6. 임오경의원: 2건
 7. 이원택의원: 2건
 8. 조계원의원: 2건
 9. 김미애의원: 2건
10. 김성원의원: 2건

=== 실제 의안 예시 ===
1. 우주항공청의 설치 및 운영에 관한 특별법 일부개정법률안(서천호의원 등 18인)
   제안자: 서천호의원 등 18인
   제안일: 2025-06-17

2. 남북교류협력에 관한 법률 일부개정법률안(김준형의원 등 11인)
   제안자: 김준형의원 등 11인
   제안일: 2025-06-11

3. 정보통신망 이용촉진 및 정보보호 등에 관한 법률 일부개정법률안(조인철의원 등 11인)
   제안자: 조인철의원 등 11인
   제안일: 2025-05-30

4. 게임산업진흥에 관한 법률 일부개정법률안(양문석의원 등 11인)
   제안자: 양문석의원 등 11인
   제안일: 2025-05-29

5. 법원조직법 일부개정법률안(박범계의원 등 10인)
   제안자: 박범계의원 등 10인
   제안일: 2025-05-23

=== 추가 통계 정보 ===
📈 의안 처리 현황:
- 원안가결: 217건
- 수정가결: 45건
- 철회: 38건

📅 연도별 의안 발의 현황:
- 2024년: 171건
- 2025년: 129건


## 2.2 정책 분야별 분석

In [15]:
policy_map = {
    '경제': ['경제', '산업', '기업', '투자', '금융', '무역', '상업', '시장', '예산', '세금', '세제'],
    '사회복지': ['복지', '사회', '보장', '지원', '급여', '연금', '의료', '건강', '보험', '돌봄'],
    '교육': ['교육', '학교', '대학', '학생', '교사', '학습', '연구', '도서관', '과학기술'],
    '환경': ['환경', '생태', '기후', '오염', '청정', '에너지', '탄소', '재생', '자연'],
    '국방안보': ['국방', '안보', '안전', '군사', '방위', '테러', '사이버', '정보보호'],
    '문화체육': ['문화', '체육', '예술', '관광', '스포츠', '영화', '음악', '방송', '언론'],
    '농업': ['농업', '농촌', '농민', '축산', '어업', '식품', '농산물', '축산물'],
    '교통': ['교통', '도로', '철도', '항공', '해운', '버스', '지하철', '교통비'],
    '외교': ['외교', '국제', '조약', '협정', '대외', '외국', '글로벌', '국경'],
    '법무': ['법', '법률', '사법', '재판', '검찰', '변호사', '인권', '형사', '민사']
}

def calc_score(bill_name, keywords):
    if pd.isna(bill_name):
        return 0
    return sum(1 for keyword in keywords if keyword in bill_name)

def calc_policy_scores(df):
    print("=== pandas를 활용한 의원별 정책 집중도 계산 ===")

    for area, keywords in policy_map.items():
        df[f'score_{area}'] = df['BILL_NM'].apply(
            lambda x: calc_score(x, keywords)
        )

    score_cols = [f'score_{area}' for area in policy_map.keys()]
    scores = df.groupby('proposer')[score_cols].sum()

    scores.columns = [col.replace('score_', '') for col in scores.columns]

    print(f"✅ {len(scores)}명 의원의 정책 점수 계산 완료")

    return scores

heatmap_df = calc_policy_scores(active_df.copy())

dict_keys(['경제', '사회복지', '교육', '환경', '국방안보', '문화체육', '농업', '교통', '외교', '법무'])
=== pandas를 활용한 의원별 정책 집중도 계산 ===
✅ 15명 의원의 정책 점수 계산 완료


# 3. 데이터 시각화 (Data Visualization)


In [16]:
def create_heatmap(df, title="의원별 정책 분야 집중도"):
    df_display = df.copy()
    df_display.index = [name[:13] + '..' if len(name) > 15 else name for name in df_display.index]

    fig = go.Figure(data=go.Heatmap(
        z=df_display.values,
        x=df_display.columns,
        y=df_display.index,
        colorscale='Blues',
        showscale=True,
        text=df_display.values,
        texttemplate="%{text:.0f}",
        textfont={"size": 10},
        hoverongaps=False,
        hovertemplate='의원: %{y}<br>정책분야: %{x}<br>점수: %{z:.0f}<extra></extra>'
    ))

    fig.update_layout(
        title=title,
        xaxis_title="정책 분야",
        yaxis_title="의원명",
        xaxis=dict(tickangle=45),
        yaxis=dict(tickangle=0),
        width=1000,
        height=800
    )

    return fig

fig = create_heatmap(heatmap_df)
fig.show()

print("✅ 인터랙티브 히트맵 생성 완료")

✅ 인터랙티브 히트맵 생성 완료
